In [13]:
# preprocess data
import numpy as np
import pandas as pd
# from hmm import UnSupervisedIOHMM, SemiSupervisedIOHMM
# from hmm import OLS,DiscreteMNL, CrossEntropyMNL

import re
import copy 
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

from hmm import OLS,DiscreteMNL, CrossEntropyMNL
from hmm import Trust_hmm
import random
from IOHMM import UnSupervisedIOHMM
from IOHMM import OLS, DiscreteMNL, CrossEntropyMNL


In [8]:
from statsmodels.tools import add_constant

def trust_transition(model, inputs, state_src, state_dst):
#     inputs = add_constant(inputs,has_constant='add')
    prob = np.exp(model.model_transition[state_src].predict_log_proba(inputs)).reshape(model.num_states)
    return prob[state_dst]

def reliance_prob(model, inputs, state):
#     inputs = add_constant(inputs,has_constant='add')

    return np.exp(model.model_emissions[state][0].predict_log_proba(inputs)).reshape(2)


def reward(model, state, confidence,attack, w_a, w_r, c):
    inputs = np.array([[confidence, attack]])

    r = 0
    for i in range(model.num_states):

        r += trust_transition(model, inputs, state, i)* (reliance_prob(model, inputs,i)[0]*w_r +
                                                         reliance_prob(model, inputs,i)[1]*w_a - attack*c)
    return r
        
def reward_belief(model, belief, confidence,attack, w_a, w_r, c):
    r = 0
    for i in range(belief.shape[0]):
        r += belief[i] * reward(model, i, confidence, attack, w_a, w_r, c)
    return r

def EU_max(model, belief, confidence, attack, l, w_a, w_r, c,max_length=3):
    if l < 1:
        print("error: l should be greated than 1")
    if l == 1:
        eu_max = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        return eu_max
    else:
        r = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        eu_max = r 
        for d in [0,1]:
            sum_ = 0
            updated_b = update_belief(model, belief, confidence, attack,reliance=d)
            for k in range(belief.shape[0]):
                b_k = belief[k]
                inputs = np.array([[ confidence, attack]])
                for i in range(model.num_states):
                    
                    sum_ += b_k*trust_transition(model, inputs, k, i)*(reliance_prob(model,inputs,i)[d]*max(0.5*(EU_max(model, updated_b,confidence=0,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)
                                                                                                              +EU_max(model, updated_b,confidence=1,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)),
                                                                                    0.5*(EU_max(model, updated_b,confidence=0,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)+EU_max(model, updated_b,confidence=1,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c))))
            eu_max += sum_
        return eu_max

def update_belief(model,belief, confidence, attack, reliance, constant = 1):
    updated_b = np.zeros_like(belief)
    inputs = np.array([[confidence, attack]])
    for j in range(belief.shape[0]):
        for i in range(belief.shape[0]):
            updated_b[j] += constant*belief[i]*trust_transition(model, inputs, state_src=i,state_dst=j)*reliance_prob(model, inputs, j)[reliance]
    return updated_b
    
def covxy(x,y):
    if type(x) == list:
        x = np.array(x)
    if type(y) == list:
        y = np.array(y)
    covxy = np.mean((x - x.mean()) * (y - y.mean()))
    return covxy

def hmm_attack_helper(model, n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        b = np.array([1/model.num_states for i in range(model.num_states)])

        tasks = assign_task(n)
        for k, t in enumerate(tasks):
            
            action = np.argmax([EU_max(model, b, t, 0, n-k, w_a, w_r, c),EU_max(model, b, t, 1, n-k, w_a, w_r, c)])
            
            if t == 0 and action==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and action==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and action == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]]) 
            b = update_belief(model, b, t,action, d)
            if d == 1:
                accept += 1
            else:
                reject += 1
#             print(accept, reject, attack)
    return w_a * accept + w_r * reject - attack*c
    
    
    
    

In [ ]:
#eval hmm-based algorithm

#1. contruct human behavior data
hmm_data = selected_data
behavior_data = []
hmm_data["reliance"] = hmm_data["reliance"].apply(lambda x: 1 if x == True else -1)
hmm_data["attack"] = hmm_data["attack"].apply(lambda x:1 if x == 0 else x)
hmm_data["human_confidence"] = hmm_data["human_confidence"].apply(lambda x:-1 if x == 0 else x)
workers = list(set(selected_data["workerId"]))
for w in workers:
    d = hmm_data.query("workerId==@w").sort_values(by=["taskId"])
    behavior_data.append(d[["human_confidence","attack", "reliance"]])
initial_columns = ["human_confidence", "attack"]
transition_columns = ["human_confidence", "attack"]
decision_columns = ["human_confidence", "attack"]

model = UnSupervisedIOHMM(num_states=3, max_EM_iter=200, EM_tol=1e-6)

model.set_models(model_emissions = [DiscreteMNL(solver='lbfgs')], 
                model_transition=CrossEntropyMNL(solver='lbfgs'),
                model_initial=CrossEntropyMNL(solver='lbfgs'))

model.set_inputs(covariates_initial = initial_columns, covariates_transition = initial_columns, covariates_emissions = [initial_columns])

model.set_outputs([['reliance']])

model.set_data(behavior_data)
model.train()
hmm_attack_helper(model, 20,0,0.5,0.15, test_humans)

In [9]:
# construct the dataset
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from sklearn.model_selection import train_test_split
def assign_task(n):
    sequence = np.random.random(n)
    sequence[sequence>0.5] = 1
    sequence[sequence<=0.5] = 0
    return sequence
    

N =1000
# analyze human reliance prob
query = pd.read_csv("query_.csv")
data = pd.read_csv("predictions_0110.csv")
data["taskId"] = data["taskId"].apply(lambda x:int(x))
data = data[data["taskId"]<16]
data["human_confidence"] = data["imageName"].apply(lambda x:query.query("Name==@x")["Confidence_Pred_val"].item())
data["human_confidence"] = data["human_confidence"].apply(lambda x:1 if x>0.5 else 0)
selected_data = data[data["attackType"]==0]
workers = list(set(selected_data["workerId"]))
h_a = []
h_n = []
l_a = []
l_n = []
for w in workers:
    d = data.query("workerId==@w")
    h_num = len(d.query("human_confidence==1")["reliance"])
    l_num = len(d.query("human_confidence==0")["reliance"])
    h_a.append(d.query("human_confidence==1 and attack==0")["reliance"].sum()/h_num)
    h_n.append(d.query("human_confidence==1 and attack==-1")["reliance"].sum()/h_num)
    l_a.append(d.query("human_confidence==0 and attack==0")["reliance"].sum()/l_num)
    l_n.append(d.query("human_confidence==0 and attack==-1")["reliance"].sum()/l_num)
mean = torch.tensor([np.mean(l_a),np.mean(h_a),np.mean(l_n),np.mean(h_n)])
var = torch.tensor([[covxy(l_a,l_a),covxy(l_a,h_a),covxy(l_a,l_n),covxy(l_a,h_n)],[covxy(h_a,l_a),covxy(h_a,h_a),covxy(h_a,l_n),covxy(h_a,h_n)],
                   [covxy(l_n,l_a),covxy(l_n,h_a),covxy(l_n,l_n),covxy(l_n,h_n)],[covxy(h_n,l_a),covxy(h_n,h_a),covxy(h_n,l_n),covxy(h_n,h_n)]])

dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())

task_pool = assign_task(n=20)

In [10]:
dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())
train_human, test_human, _, _ = train_test_split(humans,[0 for i in range(len(humans))], test_size=0.1, random_state=2023)
test_humans = [np.concatenate((i,i),0) for i in test_human]
# baselines [low_attack, high_attack, low_no, high_No]
W_a = [-1,-0.5,0]
W_r = [0.25,0.75,1]
C = np.linspace(0,1,11)
# no attack
def no_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - attack * c


def all_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = n
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - n*len(humans) * c

def half_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
   
    for h in humans:
        tasks = assign_task(n)
        p = np.random.choice([i for i in range(n)],n//2, replace =None)
        attack_p = np.zeros(n)
        attack_p[p] = 1
        for k, t in enumerate(tasks):
            if t == 0 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and attack_p[k] == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]])  
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack*c

def high_confidence_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
                attack += 1
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack * c


def eval_func(func, n, w_a, w_r, c, humans, num=50):
    u = []
    for _ in range(num):
        u.append(func(n, w_a,w_r,c,humans))
    return sum(u)/num
    
    


In [11]:
C = np.linspace(0,0.4,9)

for c in C:
    print("attack cost:---->{}".format(c))
    print("high confidence:{}".format(eval_func(high_confidence_attack_helper,20,0,0.5,c, test_humans)))
    print("half attack:{}".format(eval_func(half_attack_helper,20,0,0.5,c, test_humans)))
    print("all attack:{}".format(eval_func(all_attack_helper,20,0,0.5,c, test_humans)))
    print("no attack:{}".format(eval_func(no_attack_helper, 20,0,0.5,c, test_humans)))

attack cost:---->0.0
high confidence:295.32
half attack:325.56
all attack:423.07
no attack:227.95
attack cost:---->0.05
high confidence:270.83799999999997
half attack:300.05
all attack:371.96
no attack:229.37
attack cost:---->0.1
high confidence:244.45800000000006
half attack:274.95
all attack:322.75
no attack:229.59
attack cost:---->0.15000000000000002
high confidence:217.988
half attack:250.32
all attack:270.0
no attack:228.47
attack cost:---->0.2
high confidence:191.23600000000005
half attack:224.3
all attack:218.57
no attack:229.8
attack cost:---->0.25
high confidence:170.5
half attack:197.73
all attack:169.14
no attack:230.74
attack cost:---->0.30000000000000004
high confidence:143.58199999999997
half attack:171.91999999999996
all attack:117.17999999999998
no attack:229.0
attack cost:---->0.35000000000000003
high confidence:116.987
half attack:148.23
all attack:66.25999999999999
no attack:229.41
attack cost:---->0.4
high confidence:92.40800000000003
half attack:121.19
all attack:1

In [365]:
var

tensor([[ 0.0143,  0.0016, -0.0003,  0.0133],
        [ 0.0016,  0.0487,  0.0315, -0.0071],
        [-0.0003,  0.0315,  0.0792,  0.0026],
        [ 0.0133, -0.0071,  0.0026,  0.0508]], dtype=torch.float64)

In [321]:
d.query("human_confidence==0 and attack==-1")["reliance"].sum()

6

In [319]:
d.query("human_confidence==1 and attack==-1")["reliance"]

157     True
2316    True
2987    True
4349    True
Name: reliance, dtype: bool

In [320]:
d.query("human_confidence==1 and attack==0")["reliance"]

486     False
1829    False
1838    False
4263    False
Name: reliance, dtype: bool